In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
delitos = pd.read_csv('../../data/clean/DelitosTipo1/DelitosTipo1-2010-2020.csv')
desempleo = pd.read_csv('../../data/clean/Tasa de Desempleos/tasa_de_desempleos_y_mas_limpio.csv')


In [14]:
# Delete unnecesary column
del delitos["Unnamed: 0"]

# Rename column so it has the same name on both dfs
delitos.rename(columns = {'Distrito':'Municipio'}, inplace = True)

# Hay que tener el mismo range de datos en ambos dfs, desde 2010 hasta 2020
delitos.drop(delitos[(delitos['Date'] < '2010-01')].index, inplace=True)
delitos.drop(delitos[(delitos['Date'] > '2020-08')].index, inplace=True)

delitos

,Municipio,Tipo I,Ases.,Viol.,Robo,Agr. Grave,Esc.,Apr. I,H. Auto,Date
0,Adjuntas,34,0.0,0.0,0.0,1.0,16.0,16,1.0,2010-01-01
1,Ponce,262,11.0,0.0,28.0,27.0,50.0,125,21.0,2010-01-01
2,Peñuelas,12,0.0,1.0,2.0,1.0,2.0,5,1.0,2010-01-01
3,Patillas,17,0.0,0.0,1.0,3.0,8.0,5,0.0,2010-01-01
4,Orocovis,34,0.0,0.0,0.0,2.0,25.0,6,1.0,2010-01-01
...,...,...,...,...,...,...,...,...,...,...
9901,Naguabo,60,0.0,0.0,5.0,17.0,5.0,31,2.0,2020-07-01
9902,Naranjito,99,1.0,1.0,8.0,25.0,12.0,40,12.0,2020-07-01
9903,Orocovis,34,0.0,1.0,1.0,6.0,8.0,16,2.0,2020-07-01
9904,Patillas,34,0.0,0.0,1.0,6.0,11.0,15,1.0,2020-07-01


In [15]:
# Cambiar el nombre de la columna de los municipios
desempleo.rename(columns = {'Municipio o Area':'Municipio'}, inplace = True)

# Tengo que eliminar pueblos que contengan "-" porque estos incluyen "Areas" y 
# solo queremos evaluar los municipios individualmente
desempleo = desempleo[desempleo["Municipio"].str.contains("-|Puerto Rico|, PR") == False]
desempleo["Municipio"] = desempleo["Municipio"].str.replace(' NSA', '')

# Elimina toda data que sea antes del '2010'
desempleo.drop(desempleo[(desempleo['Date'] < '2010-01-01')].index, inplace=True)
desempleo.reset_index(drop=True,inplace=True)

# Ahora, hay que eliminar el "-01" de cada fecha para eliminar el dia
desempleo['Date'] = desempleo['Date'].str.slice(0, 7)

desempleo

,Date,Municipio,Tasa de Desempleo,Num. Personas Desempleadas,Num. Personas Empleadas,Num. Personas Grupo Trabajador
0,2010-01,Mayagüez,17.3,5055.0,23409.0,28291.0
1,2010-01,Arecibo,15.9,4972.0,26195.0,27959.0
2,2010-01,Yauco,15.3,2640.0,13204.0,12314.0
3,2010-01,Barceloneta,19.1,1582.0,5571.0,6711.0
4,2010-01,Orocovis,23.8,1206.0,4138.0,6018.0
...,...,...,...,...,...,...
9823,2020-08,Quebradillas,9.5,567.0,5413.0,5980.0
9824,2020-08,Manatí,8.5,932.0,10056.0,10988.0
9825,2020-08,Luquillo,10.4,622.0,5343.0,5965.0
9826,2020-08,Culebra,8.0,75.0,858.0,933.0


* Nota:
Tras analizar la data, nos dimos cuenta de que hay municipios que se repiten. Por ejemplo, para Mayagüez en enero-2010, hay una taza de desempleo de 16.9. Sin embargo, hay otro valor que se llama "Mayagüez, PR" que tiene otra taza de desempleo diferente de 17.6. Lo que trato de decir es que el mismo municipio está escrito con el mismo nombre y con valores distintos. Por eso se tomo la desición de eliminar aquellos que contenian el (", PR") en su nombre.

In [16]:
# After cleaning the data, I need to create a df that has both "desempleo" and "delitos" dfs

# Crear un df que haga match del municipio y el date con el resto de las columnas
df = pd.merge(desempleo, delitos, on=['Date', 'Municipio'])
# df["Fecha/Municipio"] = df['Municipio']+"+"+ df["Date"]
df

,Date,Municipio,Tasa de Desempleo,Num. Personas Desempleadas,Num. Personas Empleadas,Num. Personas Grupo Trabajador,Tipo I,Ases.,Viol.,Robo,Agr. Grave,Esc.,Apr. I,H. Auto


In [17]:
# Se puede hacer una correlacion entre taza
# Calcula la correlación entre la cantidad de delitos y la cantidad de personas desempleadas para cada pueblo
correlacion = df.groupby('Municipio')['Tipo I', 'Tasa de Desempleo'].corr().iloc[0::2, -1].reset_index()

heatmap_data = correlacion.pivot('Municipio', 'level_1', 'Tasa de Desempleo')

plt.figure(figsize=(10, 15))
plt.imshow(heatmap_data, cmap='coolwarm')
plt.colorbar()
plt.title('Correlación entre delitos y desempleo por pueblo')
plt.xlabel('Fecha')
plt.ylabel('Pueblo')
plt.xticks(range(len(heatmap_data.columns)), heatmap_data.columns, rotation=45)
plt.yticks(range(len(heatmap_data.index)), heatmap_data.index)
plt.tight_layout()
plt.show()

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [ ]:
# # Crea el scatterplot de correlación
plt.figure(figsize=(8, 6))
plt.scatter(correlacion['level_1'], correlacion['Tasa de Desempleo'])
plt.title('Correlación entre delitos y desempleo por pueblo')
plt.xlabel('Cantidad de delitos')
plt.ylabel('Cantidad de desempleados')
plt.grid(True)
plt.show()